1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이스베이스에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달

In [1]:
%pip install -qU pypdf langchain-community


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU langchain-text-splitters


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install chromadb
%pip install langchain
%pip install sentence-transformers


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -qU langchain langchainhub 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 문서 불러오기
#file_path = "text.pdf"
#loader = PyPDFLoader(file_path)
#pages = loader.load()  # 기본 로드 방식 사용

# 문서 쪼개기
# chunk_size: 문서를 쪼개는 단위
# chunk_overlap: 문서를 쪼개는 단위의 겹치는 부분
# 겹치는 부분을 선언하는 이유는 원하는 문서를 가져올 확률을 높이기 위함
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
#chunks = text_splitter.split_documents(pages)

# load_and_split 사용으로 문서 불러오기와 쪼개기를 한 번에 수행
# 분할기 정의
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

# 문서 불러오기와 쪼개기를 한 번에 수행
file_path = "text.pdf"
loader = PyPDFLoader(file_path)
chunks = loader.load_and_split(text_splitter=text_splitter)


# 임베딩
load_dotenv()
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [15]:
%pip install langchain-pinecone


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:


# 25.03.25 현재 chroma db는 python 3.13 버전에서 작동하지 않음
# 벡터 데이터베이스 생성
# chroma db는 인 메모리 벡터 데이터베이스
# database = Chroma.from_documents(
#     documents=chunks,
#     embedding=embeddings,
#     persist_directory="chroma_db",
#     collection_name="tax_collection"
# )

from langchain_pinecone import PineconeEmbeddings
# pinecone 으로 변경
# pinecone 는 클라우드 기반 벡터 데이터베이스
# pinecone console에서 데이터베이스 index 생성
# 데이터베이스 index 생성 후 데이터베이스 연결
database = Pinecone.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name="tax_collection"
)









ImportError: cannot import name 'Pinecone' from 'langchain_community.vectorstores' (/Users/jys/code/study/study-llm-application/venv310/lib/python3.10/site-packages/langchain_community/vectorstores/__init__.py)

In [6]:
# 유사도 검색
query = '연봉 3000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [ ]:
retrieved_docs

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

# 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달
# 페르소나를 전달하면 답변이 더 잘나온다고 한다.
prompt = f"""[Identity]
- 당신은 한국의 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 대답해주세요.

[Context]
{retrieved_docs}

[Question]
{query}
"""
response = llm.invoke(prompt)

In [ ]:
response.content

In [ ]:
from langchain import hub

# question-answering prompt
# 기존에 이미 있는 프롬프트 활용
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

In [16]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)


In [17]:
ai_message = qa_chain.invoke({"query": query})

In [ ]:
ai_message